<a href="https://colab.research.google.com/github/Marytem/practice_2019/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Temporary dataset preprocessing

In [0]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [0]:
train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')

In [0]:
print(type(train.data))

<class 'list'>


In [0]:
train.target

array([7, 4, 4, ..., 3, 1, 8])

In [0]:
tfidf = TfidfVectorizer(sublinear_tf=True, norm='l2', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(train.data)

In [0]:
features.shape

(11314, 1186545)

In [0]:
vectors_test = tfidf.transform(test.data)
clf = MultinomialNB(alpha=.01)
clf.fit(features, bin_tr_y)
pred = clf.predict(vectors_test)
accuracy_score(bin_tes_x, pred)

0.9641529474243229

### Model

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.regularizers import l2

Using TensorFlow backend.


In [0]:
simple_model = Sequential([Dense(100,activation='relu', input_dim=1186545, kernel_regularizer=l2(0.01)),
                           Dropout(0.3, noise_shape=None, seed=None),
                           Dense(100,activation='relu', kernel_regularizer=l2(0.01)),
                           Dropout(0.3, noise_shape=None, seed=None),
                           Dense(1,activation='sigmoid')]) 

In [0]:
simple_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
simple_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               118654600 
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 118,664,801
Trainable params: 118,664,801
Non-trainable params: 0
_________________________________________________________________


In [0]:
# ['alt.atheism',0,0
#  'comp.graphics',1,1
#  'comp.os.ms-windows.misc',2,1
#  'comp.sys.ibm.pc.hardware',3,1
#  'comp.sys.mac.hardware',4,1
#  'comp.windows.x',5,1
#  'misc.forsale',6,1
#  'rec.autos',7,0
#  'rec.motorcycles',8,0
#  'rec.sport.baseball',9.0
#  'rec.sport.hockey',10,0
#  'sci.crypt',11,1
#  'sci.electronics',12,1
#  'sci.med',13,1
#  'sci.space',14,1
#  'soc.religion.christian',15,0
#  'talk.politics.guns',16,0
#  'talk.politics.mideast',17,0
#  'talk.politics.misc',18,0
#  'talk.religion.misc',19,0]

tech = [1,2,3,4,5,6,11,12,13,14]


In [0]:
bin_tr_y = np.isin(train.target, tech).astype(np.uint8)
bin_tes_x = np.isin(test.target, tech).astype(np.uint8)

In [0]:
type(train.target)

numpy.ndarray

In [0]:
model_output= simple_model.fit(features, bin_tr_y, epochs=5, verbose=1,
                               validation_data=(vectors_test, bin_tes_x))

Train on 11314 samples, validate on 7532 samples
Epoch 1/5
11314/11314 [==============================] - 228s 20ms/step - loss: 1.1118 - acc: 0.7548 - val_loss: 1.2910 - val_acc: 0.9194
Epoch 2/5
11314/11314 [==============================] - 212s 19ms/step - loss: 1.4324 - acc: 0.9238 - val_loss: 1.5035 - val_acc: 0.9253
Epoch 3/5
11314/11314 [==============================] - 213s 19ms/step - loss: 1.4863 - acc: 0.9338 - val_loss: 1.5632 - val_acc: 0.9352
Epoch 4/5
11314/11314 [==============================] - 212s 19ms/step - loss: 1.4605 - acc: 0.9430 - val_loss: 1.5116 - val_acc: 0.9365
Epoch 5/5
11314/11314 [==============================] - 212s 19ms/step - loss: 1.4765 - acc: 0.9473 - val_loss: 1.3785 - val_acc: 0.9445


In [0]:
import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, tfjs_target_dir)

In [0]:
np.mean(model_output["acc"])

In [0]:
np.mean(model_output.hisory["val_acc"])

In [0]:
from keras.utils import to_categorical
train_target = to_categorical(train.target)
test_target = to_categorical(test.target)

In [0]:
simple_model_cat = Sequential([Dense(100,activation='relu', input_dim=1186545, kernel_regularizer=l2(0.01)),
                           Dropout(0.3, noise_shape=None, seed=None),
                           Dense(100,activation='relu', kernel_regularizer=l2(0.01)),
                           Dropout(0.3, noise_shape=None, seed=None),
                           Dense(1,activation='softmax')]) 

In [0]:
simple_model_cat.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [0]:
simple_model_cat.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 100)               118654600 
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 100)               10100     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 118,664,801
Trainable params: 118,664,801
Non-trainable params: 0
_________________________________________________________________


In [0]:
model_output= simple_model_cat.fit(features, train.target, epochs=5, verbose=1,
                               validation_data=(vectors_test, test.target))